In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import re
import os
import ast
import cv2
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
from mpl_toolkits.mplot3d import Axes3D
from datetime import date
from scipy import stats
from scipy.stats import norm,skew
from scipy.special import boxcox1p,inv_boxcox1p,boxcox
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder,StandardScaler
from sklearn.base import BaseEstimator,TransformerMixin,RegressorMixin,clone
from collections import defaultdict
import xgboost as xgb
import lightgbm as lgb
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5);
sns.set_style('whitegrid')

In [ ]:
# https://www.kaggle.com/mobasshir/recommendation-engine

In [ ]:
with open('/content/drive/My Drive/Colab/Bil573/VariableDefinitions.txt') as f:
    print(f.read())

﻿Train Customers


Information on the customers in the training set. 


'akeed_customer_id': Unique customer ID, used in train_locations and train_orders
'gender': Customer gender
'dob': Birth Year (if entered)
'status' and 'verified': Account status
'language': Chosen language
'Created_at' and 'updated_at': dates when account was created/updated


Train Locations


Each customer orders from one or more locations. Each is assigned a location number. 
'customer_id': The unique customer ID 
'location_number': Location number (most customers have one or two)
'location_type': Home, Work, Other or NA
'Latitude' and 'longitude': Not true latitude and longitude - locations have been masked, but nearby locations remain nearby in the new reference frame and can thus be used for clustering. However, not all locations are useful due to GPS errors and missing data - you may want to treat outliers separately.


Train Orders


This is a record of all orders made by customers in the train set from th

# TRAIN_FULL

In [ ]:
train_full = pd.read_csv("/content/drive/My Drive/Colab/Bil573/train_full.csv")
nRow, nCol = train_full.shape
print(f'There are {nRow} rows and {nCol} columns')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 5802400 rows and 73 columns


In [ ]:
train_full.dropna(subset=['latitude_x'], inplace=True)
train_full.dropna(subset=['longitude_x'], inplace=True)
train_full.dropna(subset=['device_type'], inplace=True)
train_full.dropna(subset=['display_orders'], inplace=True)
train_full.dropna(subset=['location_number_obj'], inplace=True)
train_full.dropna(subset=['id_obj'], inplace=True)
train_full.dropna(subset=['target'], inplace=True)

In [ ]:
train_full.loc[train_full["location_type"].isna(), 'location_type'] = 0
train_full.loc[train_full["location_type"]=="Home", 'location_type'] = 1
train_full.loc[train_full["location_type"]=="Work", 'location_type'] = 2
train_full.loc[train_full["location_type"]=="Other", 'location_type'] = 3

In [ ]:
train_full.loc[train_full["gender"].isna(), 'gender'] = 0
train_full.loc[train_full["gender"]=="Male", 'gender'] = 1
train_full.loc[train_full["gender"]=="Female", 'gender'] = 2

In [ ]:
train_full.loc[train_full["vendor_category_id"]==2, 'vendor_category_id'] = 0
train_full.loc[train_full["vendor_category_id"]==3, 'vendor_category_id'] = 1

In [ ]:
train_full["vendor_rating"] = train_full["vendor_rating"] - 3.8

In [ ]:
train_full=train_full[train_full['is_open']!=0]

In [ ]:
features_to_drop=["created_at_x", "updated_at_x", "authentication_id", "vendor_category_en", "is_open", "OpeningTime", "OpeningTime2", "sunday_from_time1", "sunday_to_time1",
                  "sunday_from_time2", "sunday_to_time2", "monday_from_time1", "monday_to_time1", "monday_from_time2", "monday_to_time2", "tuesday_from_time1",
                  "tuesday_to_time1", "tuesday_from_time2", "tuesday_to_time2", "wednesday_from_time1", "wednesday_to_time1", "wednesday_from_time2",
                  "wednesday_to_time2", "thursday_from_time1", "thursday_to_time1", "thursday_from_time2", "thursday_to_time2", "friday_from_time1",
                  "friday_to_time1", "friday_from_time2", "friday_to_time2", "saturday_from_time1", "saturday_to_time1", "saturday_from_time2", "saturday_to_time2",
                  "commission", "is_akeed_delivering", "status_y", "verified_y", "language", "primary_tags", "open_close_flags", "vendor_tag", "vendor_tag_name",
                  "one_click_vendor", "country_id", "city_id", "created_at_y", "updated_at_y", "display_orders", "CID X LOC_NUM X VENDOR"]

for feature in features_to_drop:
  train_full=train_full.drop([feature],axis=1)

In [ ]:
train_full['target'].value_counts()

0    4863748
1      67782
Name: target, dtype: int64

In [ ]:
train_full['target'].isna().sum()

0

In [ ]:
train_full.isna().sum().sum()

0

In [ ]:
train_full.shape

(4931530, 22)

In [ ]:
train_full.head(3)

,customer_id,gender,status_x,verified_x,location_number,location_type,latitude_x,longitude_x,id,latitude_y,longitude_y,vendor_category_id,delivery_charge,serving_distance,prepration_time,discount_percentage,rank,vendor_rating,device_type,location_number_obj,id_obj,target
0,TCHWPBT,1,1,1,0,2,-96.44,-67.2,4,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,0,4,0
1,TCHWPBT,1,1,1,0,2,-96.44,-67.2,13,-0.4717,0.7446,0.0,0.7,5.0,14,0.0,11,0.9,3,0,13,0
2,TCHWPBT,1,1,1,0,2,-96.44,-67.2,20,-0.4075,0.6436,0.0,0.0,8.0,19,0.0,1,0.7,3,0,20,0


In [ ]:
train_full.describe()

,status_x,verified_x,location_number,latitude_x,longitude_x,id,latitude_y,longitude_y,vendor_category_id,delivery_charge,serving_distance,prepration_time,discount_percentage,rank,vendor_rating,device_type,location_number_obj,id_obj,target
count,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06,4.931530e+06
mean,9.991727e-01,9.715261e-01,7.501120e-01,2.355730e+00,-2.515269e+01,3.011529e+02,2.263520e+00,8.948918e-01,9.411765e-02,3.870588e-01,1.141176e+01,1.360000e+01,1.294118e+00,9.705882e+00,5.811765e-01,2.976471e+00,7.501120e-01,3.011529e+02,1.374462e-02
std,2.875144e-02,1.663225e-01,1.329800e+00,2.276045e+01,3.695347e+01,2.553900e+02,2.220222e+01,4.739869e+00,2.919924e-01,3.480326e-01,4.246228e+00,2.987031e+00,6.780290e+00,3.356552e+00,2.089155e-01,2.156506e-01,1.329800e+00,2.553900e+02,1.164290e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.038000e+03,-1.344000e+02,4.000000e+00,-1.588000e+00,-1.449000e+00,0.000000e+00,0.000000e+00,2.000000e+00,5.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+00,0.000000e+00,-5.034000e-01,-7.850000e+01,9.200000e+01,-7.120000e-01,8.770000e-02,0.000000e+00,0.000000e+00,8.000000e+00,1.000000e+01,0.000000e+00,1.100000e+01,4.000000e-01,3.000000e+00,0.000000e+00,9.200000e+01,0.000000e+00
50%,1.000000e+00,1.000000e+00,0.000000e+00,-8.826000e-02,2.106500e-02,2.070000e+02,-4.922000e-01,5.273000e-01,0.000000e+00,7.000000e-01,1.500000e+01,1.500000e+01,0.000000e+00,1.100000e+01,6.000000e-01,3.000000e+00,0.000000e+00,2.070000e+02,0.000000e+00
75%,1.000000e+00,1.000000e+00,1.000000e+00,2.610000e-01,5.130000e-01,4.010000e+02,5.750000e-03,6.560000e-01,0.000000e+00,7.000000e-01,1.500000e+01,1.500000e+01,0.000000e+00,1.100000e+01,7.000000e-01,3.000000e+00,1.000000e+00,4.010000e+02,0.000000e+00
max,1.000000e+00,1.000000e+00,2.900000e+01,4.538000e+02,4.534000e+01,9.070000e+02,2.052000e+02,4.422000e+01,1.000000e+00,7.000000e-01,1.500000e+01,2.100000e+01,5.000000e+01,1.100000e+01,1.000000e+00,3.000000e+00,2.900000e+01,9.070000e+02,1.000000e+00


In [ ]:
train_full.to_csv("/content/drive/My Drive/Colab/Bil573/train_full_processed.csv")

# PREPARE DATASET

In [ ]:
# Üstte 3 train den sütunlar silindi
# 3 parçayı birleştir
# train test ayır
# label 0 1

In [ ]:
# Customer için
# Girdiyi restaurantlara böl
# her birini yüzde 70 olasılıkla trainigne 30 ile teste at

In [ ]:
train_full = pd.read_csv("/content/drive/My Drive/Colab/Bil573/train_full_processed.csv")

In [ ]:
train_full.columns

Index(['Unnamed: 0', 'customer_id', 'gender', 'status_x', 'verified_x',
       'location_number', 'location_type', 'latitude_x', 'longitude_x', 'id',
       'latitude_y', 'longitude_y', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'prepration_time', 'discount_percentage', 'rank',
       'vendor_rating', 'device_type', 'location_number_obj', 'id_obj',
       'target'],
      dtype='object')

In [ ]:
lbl=LabelEncoder()
lbl.fit(train_full["customer_id"].values)
train_full["customer_id"]=lbl.transform(train_full["customer_id"].values)


In [ ]:
lbl=LabelEncoder()
lbl.fit(train_full["id"].values)
train_full["id"]=lbl.transform(train_full["id"].values)

In [ ]:
sum(train_full.groupby('customer_id')['id'].nunique() == 85)/len(train_full.groupby('customer_id'))

1.0

In [ ]:
"""
  Drop customers below N where customer goes to any restaurant and like it 
"""
filtered_customers_id =train_full[train_full['target']==1].groupby('customer_id')['target'].count() >= 6
filtered_customers_id_lower = filtered_customers_id.index[filtered_customers_id]

In [ ]:
"""
  Drop customers below N where customer goes to any restaurant and does not like it 
"""
filtered_customers_id =train_full[train_full['target']==0].groupby('customer_id')['target'].count() > 10
filtered_customers_id_upper = filtered_customers_id.index[filtered_customers_id]

In [ ]:
"""
    Concatenate upper and lower(A B kesişim)
"""
train_full_filtered = train_full[train_full['customer_id'].isin(filtered_customers_id_upper)]#.groupby('customer_id')['target'].count()
train_full_filtered = train_full_filtered[train_full_filtered['customer_id'].isin(filtered_customers_id_lower)]#.groupby('customer_id')['target'].count()

In [ ]:
number_of_customer = train_full_filtered["customer_id"].nunique()


In [ ]:
number_of_customer

2854

In [ ]:
train_full_filtered["customer_id"].unique()

array([33977,  1454, 17716, ...,  1783,  9151, 20241])

In [ ]:
number_of_restaurant = train_full_filtered["id"].nunique()

In [ ]:
#df = train_full_filtered[train_full_filtered['customer_id']==5]
#df[df["id"] == 8]

In [ ]:
train_full.shape

(4931530, 23)

In [ ]:
df_2 = train_full_filtered[train_full_filtered['customer_id'] ==13]
#indices = train_full_filtered[train_full_filtered['customer_id'] ==33977][1]]['target'] == 1#.groupby("id")
df_2[df_2['target'] == 1]


,Unnamed: 0,customer_id,gender,status_x,verified_x,location_number,location_type,latitude_x,longitude_x,id,latitude_y,longitude_y,vendor_category_id,delivery_charge,serving_distance,prepration_time,discount_percentage,rank,vendor_rating,device_type,location_number_obj,id_obj,target
311,370,33977,1,1,1,0,1,-0.17550,-78.5600,56,0.00575,0.4822,0.0,0.7,15.0,15,0.0,11,0.9,3,0,303,1
344,404,33977,1,1,1,1,1,0.19120,-78.6000,4,0.48070,0.5527,0.0,0.7,15.0,10,0.0,11,0.6,3,1,28,1
345,405,33977,1,1,1,1,1,0.19120,-78.6000,5,-0.49460,0.7430,0.0,0.7,6.0,17,0.0,11,0.8,3,1,33,1
398,472,33977,1,1,1,1,1,0.19120,-78.6000,58,-0.01683,0.5146,0.0,0.7,15.0,13,0.0,11,1.0,3,1,310,1
515,605,33977,1,1,1,3,1,0.04245,-78.6000,5,-0.49460,0.7430,0.0,0.7,6.0,17,0.0,11,0.8,3,3,33,1
521,611,33977,1,1,1,3,1,0.04245,-78.6000,11,-0.60060,0.7560,0.0,0.7,5.0,15,0.0,11,0.8,3,3,75,1
541,635,33977,1,1,1,3,1,0.04245,-78.6000,31,-0.60000,0.7563,0.0,0.0,6.0,13,0.0,11,0.7,3,3,159,1
565,669,33977,1,1,1,3,1,0.04245,-78.6000,55,-0.80700,0.0914,0.0,0.7,15.0,15,0.0,11,0.6,3,3,300,1
595,700,33977,1,1,1,4,1,-0.63100,-78.5000,0,-0.58840,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,4,4,1
600,705,33977,1,1,1,4,1,-0.63100,-78.5000,5,-0.49460,0.7430,0.0,0.7,6.0,17,0.0,11,0.8,3,4,33,1


In [ ]:
df_2[df_2['id'] == 0]

,Unnamed: 0,customer_id,gender,status_x,verified_x,location_number,location_type,latitude_x,longitude_x,id,latitude_y,longitude_y,vendor_category_id,delivery_charge,serving_distance,prepration_time,discount_percentage,rank,vendor_rating,device_type,location_number_obj,id_obj,target
255,300,33977,1,1,1,0,1,-0.17550,-78.5600,0,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,0,4,0
340,400,33977,1,1,1,1,1,0.19120,-78.6000,0,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,1,4,0
425,500,33977,1,1,1,2,1,-96.44000,-67.2000,0,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,2,4,0
510,600,33977,1,1,1,3,1,0.04245,-78.6000,0,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,3,4,0
595,700,33977,1,1,1,4,1,-0.63100,-78.5000,0,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,4,4,1
680,800,33977,1,1,1,5,0,-0.17830,0.4927,0,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,5,4,0
765,900,33977,1,1,1,6,0,-1.09800,-78.4400,0,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,6,4,0
850,1000,33977,1,1,1,7,0,-0.00603,0.5460,0,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,7,4,0
935,1100,33977,1,1,1,8,0,-0.06210,0.4392,0,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,8,4,0
1020,1200,33977,1,1,1,9,3,0.19510,0.5425,0,-0.5884,0.7544,0.0,0.0,6.0,15,0.0,11,0.6,3,9,4,0


In [ ]:
"""
    Create customer history matrix and TODO:add customer unique name to a list
    Split data set balanced %50 probability + liked,%80 - not liked
"""
customer_history = np.zeros((number_of_customer,number_of_restaurant+1))#id ,location number
list_customer_id = train_full_filtered["customer_id"].unique()

for i in range(len(list_customer_id)):
   df = train_full_filtered[train_full_filtered["customer_id"] == list_customer_id[i]]
   pos = df[df["target"] == 1]["id"]
   neg = df[df["target"] == 0]["id"]

   customer_history[i,0] = list_customer_id[i]
   for p in pos:
     if customer_history[i,p+1] == 0:
        customer_history[i,p+1] = customer_history[i,p+1] + 1

   for n in neg:
     if customer_history[i,n+1] == 0:
        customer_history[i,n+1] = customer_history[i,n+1] - 1

list_customer_id 2854


In [ ]:
customer_history[1][0]

1454.0

In [ ]:
import random

training_matrix = np.copy(customer_history)#label
for i in range(customer_history.shape[0]):
    for j in range(1,customer_history.shape[1]):
        #Split dataset train and validation
        rand = random.uniform(0, 1)
        if customer_history[i][j] == 1 and rand > 0.60:#%10 validation,%30 test 
           training_matrix[i][j] = 0
        if customer_history[i][j] == -1 and rand > 0.70:#%10 validation,%20 test
           training_matrix[i][j] = 0

In [ ]:
#list_customer_id
training_data = []
training_label = []
test_data = []
test_labels = []
validation_data = []
validation_label = []
training_customers = []
corr = []
for j in range(1,training_matrix.shape[1]):
  
    training_data.append([])
    training_label.append([])
    test_labels.append([])
    test_data.append([])
    validation_data.append([])
    validation_label.append([])

    corr.append([])
    for i in range(training_matrix.shape[0]):
       #Training data 
       if training_matrix[i,j] != 0:
            temp = training_matrix[i,j]
            copy_matrix = np.copy(training_matrix[i,:])
            training_data[j-1].append(copy_matrix)
            training_data[j-1][-1][j] = 0
            training_label[j-1].append(temp)

            corr[j-1].append(i)
        #Test data
       else:
            temp = customer_history[i,j]
            rand = random.uniform(0, 1)
            copy_matrix = np.copy(training_matrix[i,:])

            if temp == 1 and rand < 0.25:#Probability of a data in validation for  liked 
               validation_data[j-1].append(copy_matrix)
               validation_label[j-1].append(temp)
            if temp == 1 and rand >= 0.25:#Prob of in test for liked
               test_data[j-1].append(copy_matrix)
               test_labels[j-1].append(temp)
            if temp == -1 and rand < 0.33:#Probability of a data in validation for not  liked 
               validation_data[j-1].append(copy_matrix)
               validation_label[j-1].append(temp)
            if temp == -1 and rand >= 0.33:#Probability of a data in test for not  liked 
               test_data[j-1].append(copy_matrix)
               test_labels[j-1].append(temp)
            
           

In [ ]:
column_name = []
for i in range(85):
   name = "rest_" + str(i)
   column_name.append(name)
for d in train_full_filtered.columns[1:].tolist():
   column_name.append(d)

column_name ['rest_0', 'rest_1', 'rest_2', 'rest_3', 'rest_4', 'rest_5', 'rest_6', 'rest_7', 'rest_8', 'rest_9', 'rest_10', 'rest_11', 'rest_12', 'rest_13', 'rest_14', 'rest_15', 'rest_16', 'rest_17', 'rest_18', 'rest_19', 'rest_20', 'rest_21', 'rest_22', 'rest_23', 'rest_24', 'rest_25', 'rest_26', 'rest_27', 'rest_28', 'rest_29', 'rest_30', 'rest_31', 'rest_32', 'rest_33', 'rest_34', 'rest_35', 'rest_36', 'rest_37', 'rest_38', 'rest_39', 'rest_40', 'rest_41', 'rest_42', 'rest_43', 'rest_44', 'rest_45', 'rest_46', 'rest_47', 'rest_48', 'rest_49', 'rest_50', 'rest_51', 'rest_52', 'rest_53', 'rest_54', 'rest_55', 'rest_56', 'rest_57', 'rest_58', 'rest_59', 'rest_60', 'rest_61', 'rest_62', 'rest_63', 'rest_64', 'rest_65', 'rest_66', 'rest_67', 'rest_68', 'rest_69', 'rest_70', 'rest_71', 'rest_72', 'rest_73', 'rest_74', 'rest_75', 'rest_76', 'rest_77', 'rest_78', 'rest_79', 'rest_80', 'rest_81', 'rest_82', 'rest_83', 'rest_84', 'customer_id', 'gender', 'status_x', 'verified_x', 'location_n

In [ ]:
def generate_dataset(data,labels,path,train_full_filtered):
  for i in range(len(data)):
   test_csv = np.zeros((1,85+number_of_attributes))
   for j in range(len(data[i])):
      df = train_full_filtered[train_full_filtered["customer_id"] == int(data[i][j][0])]
      df = df[df["id"] == i]
      restorant_preference = labels[i][j]
      preference_list = np.asarray(data[i][j][1:])
      if restorant_preference == -1:
        location_number = df[df["target"] == 0]
        preference_list = np.concatenate((preference_list,location_number.values[0][1:]),axis=0)
        preference_list = np.expand_dims(preference_list,axis=0)
        test_csv = np.append(test_csv,preference_list,axis=0)
      if restorant_preference == 1:
        location_number = df[df["target"] == 1]
        preference_list = np.concatenate((preference_list,location_number.values[0][1:]),axis=0)
        preference_list = np.expand_dims(preference_list,axis=0)
        test_csv = np.append(test_csv,preference_list,axis=0)
   output_csv = pd.DataFrame(data=test_csv, columns=column_name)
   output_csv.to_csv("/content/drive/MyDrive/Bil573/"+path+"/"+"restaurant_"+str(i)+".csv")
   print("df",test_csv.shape)

In [ ]:
generate_dataset(training_data,training_label,"train_data",train_full_filtered)

In [ ]:
generate_dataset(test_data,test_labels,"test_data",train_full_filtered)

In [ ]:
generate_dataset(validation_data,validation_label,"validation_data",train_full_filtered)

df (274, 107)
df (301, 107)
df (282, 107)
df (319, 107)
df (278, 107)
df (264, 107)
df (271, 107)
df (281, 107)
df (291, 107)
df (299, 107)
df (289, 107)
df (289, 107)
df (302, 107)
df (277, 107)
df (267, 107)
df (281, 107)
df (302, 107)
df (263, 107)
df (303, 107)
df (288, 107)
df (270, 107)
df (281, 107)
df (268, 107)
df (288, 107)
df (305, 107)
df (249, 107)
df (304, 107)
df (281, 107)
df (277, 107)
df (271, 107)
df (272, 107)
df (258, 107)
df (275, 107)
df (295, 107)
df (315, 107)
df (257, 107)
df (282, 107)
df (280, 107)
df (242, 107)
df (315, 107)
df (280, 107)
df (285, 107)
df (290, 107)
df (268, 107)
df (299, 107)
df (304, 107)
df (261, 107)
df (298, 107)
df (283, 107)
df (307, 107)
df (301, 107)
df (291, 107)
df (264, 107)
df (275, 107)
df (291, 107)
df (256, 107)
df (287, 107)
df (280, 107)
df (289, 107)
df (289, 107)
df (264, 107)
df (310, 107)
df (262, 107)
df (273, 107)
df (277, 107)
df (273, 107)
df (269, 107)
df (260, 107)
df (322, 107)
df (251, 107)
df (294, 107)
df (30

In [ ]:
explicit_df = df[['customer_id','id','restaurent_rating']]
explicit_df.columns = ['customer_id','restaurent_id','restaurent_rating']

KeyError: ignored

In [ ]:
explicit_df.shape

In [ ]:
explicit_df.info()

In [ ]:
print('Dataset shape: {}'.format(explicit_df.shape))
print('-Dataset examples-')
print(explicit_df.iloc[::20000, :])

In [ ]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = explicit_df['restaurent_rating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / explicit_df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} ratings'.format(explicit_df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [ ]:
# Number of ratings per book
data = explicit_df.groupby('restaurent_id')['restaurent_rating'].count().clip(upper=100)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 100,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per Restaurent (Clipped at 100)',
                   xaxis = dict(title = 'Number of Ratings Per Restaurent'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [ ]:
explicit_df.groupby('restaurent_id')['restaurent_rating'].count().reset_index().sort_values('restaurent_rating', ascending=False)[:10]

In [ ]:
# Number of ratings per user
data = explicit_df.groupby('customer_id')['restaurent_rating'].count().clip(upper=100)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 100,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User (Clipped at 100)',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [ ]:
explicit_df.groupby('customer_id')['restaurent_rating'].count().reset_index().sort_values('restaurent_rating', ascending=False)[:10]

# VISUALIZE DATASET

In [ ]:
# BURAYI DUZELT
fig, ax = plt.subplots(1, 2)
train_full['gender'].hist(ax=ax[0], color='yellow')
train_full['location_type'].hist(ax=ax[1])

In [ ]:
train_full['rank'].hist()

In [ ]:
train_full['prepration_time'].hist()

In [ ]:
explicit_df.restaurent_rating.unique()

In [ ]:
df = explicit_df

In [ ]:
in_ratings = 5
filter_items = df['restaurent_id'].value_counts() > min_ratings
filter_items = filter_items[filter_items].index.tolist()

min_user_ratings = 5
filter_users = df['customer_id'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['restaurent_id'].isin(filter_items)) & (df['customer_id'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_new[['customer_id', 'restaurent_id', 'restaurent_rating']], reader)

In [ ]:
benchmark = []

algorithms = [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]

print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')

In [ ]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [ ]:
surprise_results

# FEATURE ENGINEERING

# TRAINING

In [ ]:
# softmax kullan
# 2li sınıflandırma
# Bu kullanıcı bu restauranı sever mi
# Belli yakınlıktakiler için yüzde bul

In [ ]:
# LightGBM
# KNN
# SVD
# K-means
# Random Tree
# Random Forest

# TEST

# ESKİ KODLAR

In [ ]:
test.index=test.index+3000
df=pd.concat([train.drop("revenue",axis=1),test]).drop("id", axis=1) #drop id
y_train=train["revenue"]
df.shape

In [ ]:
def visualize_distribution(y):
    sns.distplot(y,fit=norm)
    mu,sigma=norm.fit(y)
    plt.legend(["Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f})".format(mu,sigma)])
    plt.title("Distribution of revenue")
    plt.ylabel("Frequency")
    plt.show()
    
def visualize_probplot(y):
    stats.probplot(y,plot=plt)
    plt.show()

In [ ]:
visualize_distribution(y_train)
visualize_probplot(y_train)

##### We want our model to perform as good as possible and in order to do so the target variable of y_train must be as close as possible to the normal distribution (and its probability plot as close as linear as possible

In [ ]:
# boxcox transformation with coefficient 0.2 makes the probability plot closer to linear
y_train=boxcox1p(y_train,0.2)
visualize_distribution(y_train)
visualize_probplot(y_train)

### Fix Features

In [ ]:
features_to_fix=["belongs_to_collection", "genres", "production_companies", "production_countries",\
                 "Keywords"]

for feature in features_to_fix:
    df.loc[df[feature].notnull(),feature]=df.loc[df[feature].notnull(),feature].apply(lambda x : ast.literal_eval(x)).apply(lambda x : [y["name"] for y in x])

df.loc[df["cast"].notnull(),"cast"]=df.loc[df["cast"].notnull(),"cast"].apply(lambda x : ast.literal_eval(x))
df.loc[df["crew"].notnull(),"crew"]=df.loc[df["crew"].notnull(),"crew"].apply(lambda x : ast.literal_eval(x))

### create new features

In [ ]:
df["cast_len"] = df.loc[df["cast"].notnull(),"cast"].apply(lambda x : len(x))
df["crew_len"] = df.loc[df["crew"].notnull(),"crew"].apply(lambda x : len(x))

lbl=LabelEncoder()
lbl.fit(df["release_year"].values)
df["release_year"]=lbl.transform(df["release_year"].values)

### Modify cast and crew

In [ ]:
# For each film we only want to consider the six most important actors
# order is the order of importance in ascending order starting from 0
df.loc[df["cast"].notnull(),"cast"]=df.loc[df["cast"].notnull(),"cast"].apply(lambda x : [y["name"] for y in x if y["order"]<6])

In [ ]:
df["Director"]=[[] for i in range(df.shape[0])]
df["Producer"]=[[] for i in range(df.shape[0])]
df["Executive Producer"]=[[] for i in range(df.shape[0])]

df["Director"]=df.loc[df["crew"].notnull(),"crew"].apply(lambda x : [y["name"] for y in x if y["job"]=="Director"])
df["Producer"]=df.loc[df["crew"].notnull(),"crew"].apply(lambda x : [y["name"] for y in x if y["job"]=="Producer"])
df["Executive Producer"]=df.loc[df["crew"].notnull(),"crew"].apply(lambda x : [y["name"] for y in x if y["job"]=="Executive Producer"])

### delete useless features 

In [ ]:
df=df.drop(["imdb_id","original_title","overview","poster_path","tagline","status","title","spoken_languages","release_date","crew"],axis=1)

### Handle missing values

In [ ]:
mis_val=((df.isnull().sum()/df.shape[0])*100).sort_values(ascending=False)
mis_val=mis_val.drop(mis_val[mis_val==0].index)
mis_val

In [ ]:
# missing values will be replaced with empty lists
to_empty_list=["belongs_to_collection","Keywords","production_companies","production_countries",\
              "Director","Producer","Executive Producer","cast","genres"]

for feature in to_empty_list:
    df[feature] = df[feature].apply(lambda d: d if isinstance(d, list) else [])

In [ ]:
# The missing values of time and length will be replaced with 0
to_zero=["runtime","release_month","release_year","release_wd","release_quarter","release_day"]+\
["Keywords_len","production_companies_len","production_countries_len","crew_len","cast_len","genres_len",
    "tagline_word_count","overview_word_count","title_word_count"]

for feat in to_zero:
    df[feat]=df[feat].fillna(0)

In [ ]:
# add new features
df['_budget_popularity_ratio'] = df['budget']/df['popularity']
df['_releaseYear_popularity_ratio'] = df['release_year']/df['popularity']
df['_releaseYear_popularity_ratio2'] = df['popularity']/df['release_year']
df['_releaseYear_budget_ratio'] = df['release_year']/df['budget']
df['_releaseYear_budget_ratio2'] = df['budget']/df['release_year']

In [ ]:
mis_val=((df.isnull().sum()/df.shape[0])*100).sort_values(ascending=False)
mis_val=mis_val.drop(mis_val[mis_val==0].index)
mis_val

### modify the numeric features in order to make them as close as possible to the normal distribution

In [ ]:
numeric=[feat for feat in df.columns if df[feat].dtype!="object"]

skewness=df[numeric].apply(lambda x : skew(x)).sort_values(ascending=False)
skew=skewness[skewness>2.5]
skew

In [ ]:
# reduce the skewness of these features by applying a power transformation
high_skew=skew[skew>10].index
medium_skew=skew[skew<=10].index

for feat in high_skew:
    df[feat]=np.log1p(df[feat])

for feat in medium_skew:
    df[feat]=df[feat]=boxcox1p(df[feat],0.15)

skew=df[skew.index].skew()
skew

In [ ]:
plt.figure(figsize=(15,40))
for i,feat in enumerate(skew.index):
    plt.subplot(7,2,i+1)
    sns.distplot(df[feat],fit=norm)
    mu,sigma=norm.fit(df[feat])
    plt.legend(["Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f})".format(mu,sigma)])
    plt.title("Distribution of "+feat)
    plt.ylabel("Frequency")
plt.show()

In [ ]:
lbl=LabelEncoder()
lbl.fit(df["release_year"].values)
df["release_year"]=lbl.transform(df["release_year"].values)

In [ ]:
to_dummy = ["belongs_to_collection","genres","original_language","production_companies","production_countries",\
           "Keywords","cast","Director","Producer","Executive Producer"]

In [ ]:
limits=[4,0,0,35,10,40,10,5,10,12] 

for i,feat in enumerate(to_dummy):
    mlb = MultiLabelBinarizer()
    s=df[feat]
    x=pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)
    y=pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index).sum().sort_values(ascending=False)
    rare_entries=y[y<=limits[i]].index
    x=x.drop(rare_entries,axis=1)
    df=df.drop(feat,axis=1)
    df=pd.concat([df, x], axis=1, sort=False)


In [ ]:
ntrain=train.shape[0]

train=df.iloc[:ntrain,:]
test=df.iloc[ntrain:,:]
test.shape

# train

In [ ]:
model_xgb=xgb.XGBRegressor(max_depth=5,
                           learning_rate=0.1, 
                           n_estimators=2000, 
                           objective='reg:linear', 
                           gamma=1.45, 
                           verbosity=3,
                           subsample=0.7, 
                           colsample_bytree=0.8, 
                           colsample_bylevel=0.50)

In [ ]:
model_lgb=lgb.LGBMRegressor(n_estimators=10000, 
                             objective="regression", 
                             metric="rmse", 
                             num_leaves=20, 
                             min_child_samples=100,
                             learning_rate=0.01, 
                             bagging_fraction=0.8, 
                             feature_fraction=0.8, 
                             bagging_frequency=1, 
                             subsample=.9, 
                             colsample_bytree=.9,
                             use_best_model=True)

In [ ]:
model_cat = cat.CatBoostRegressor(iterations=10000,learning_rate=0.01,depth=5,eval_metric='RMSE',\
                              colsample_bylevel=0.7,
                              bagging_temperature = 0.2,
                              metric_period = None,
                              early_stopping_rounds=200)

In [ ]:
def cross_val(model):
    cr_val=np.sqrt(-cross_val_score(model,train.values,y_train.values,scoring="neg_mean_squared_log_error",cv=5))
    return cr_val

In [ ]:
def msle(y,y_pred):
    return np.sqrt(mean_squared_log_error(y,y_pred))

In [ ]:
ti=time.time()
model_lgb.fit(train.values,y_train)
print("Number of minutes of training of model_lgb = {:.2f}".format((time.time()-ti)/60))

lgb_pred_train=model_lgb.predict(train.values)
print("Mean square logarithmic error of lgb model on whole train = {:.4f}".format(msle(y_train,lgb_pred_train)))

In [ ]:
ti=time.time()
model_xgb.fit(train.values,y_train)
print("Number of minutes of training of model_xgb = {:.2f}".format((time.time()-ti)/60))

xgb_pred_train=model_xgb.predict(train.values)
print("Mean square logarithmic error of xgb model on whole train = {:.4f}".format(msle(y_train,xgb_pred_train)))

In [ ]:
ti=time.time()
model_cat.fit(train.values,y_train,verbose=False)
print("Number of minutes of training of model_cal = {:.2f}".format((time.time()-ti)/60))

cat_pred_train=model_cat.predict(train.values)
cat_pred_train[cat_pred_train<0]=0
print("Mean square logarithmic error of cat model on whole train = {:.4f}".format(msle(y_train,cat_pred_train)))

In [ ]:
c = np.array([0.333334,0.333333,0.333333])

train_pred=xgb_pred_train*c[0]+lgb_pred_train*c[1]+cat_pred_train*c[2]
print("Mean square logarithmic error of chosen model on whole train = {:.4f}".format(msle(y_train,train_pred)))

# test

In [ ]:
lgb_pred=model_lgb.predict(test)
xgb_pred=model_xgb.predict(test.values)
cat_pred=model_cat.predict(test)

In [ ]:
# apply the inverse of the tranformation we applied on y_train
pred=inv_boxcox1p((xgb_pred*c[0]+lgb_pred*c[1]+cat_pred*c[2]),0.2)

sub=pd.DataFrame({"id":np.arange(test.shape[0])+3001,"revenue":pred})
sub.to_csv("my_submission.csv",index=False)

# INTERNETTEN ALINANLAR

In [ ]:
def dataframe_split_between_null_and_not_null(df):
    not_null_df = df.dropna()
    null_df = df.drop(not_null_df.index)
    percentage = 100 * null_df.shape[0] / df.shape[0]
    print("Actual dataset ", df.shape)
    print("Null dataset ", null_df.shape)
    print("Not null dataset ", not_null_df.shape)
    print(f'Null percentage {percentage}%')
    return df, null_df, not_null_df,percentage

In [ ]:
def column_missing_state(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * mis_val / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table = mis_val_table[mis_val_table.iloc[:,1] != 0].sort_values(1, ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table.shape[0]) +
              " columns that have missing values.")
    return mis_val_table

# https://www.kaggle.com/erelin6613/pyspark-alternating-least-squares-in-action

# TRAIN_CUSTOMERS

In [ ]:
train_customers = pd.read_csv("/content/drive/My Drive/Colab/Bil573/train_customers.csv")
nRow, nCol = train_customers.shape
print(f'There are {nRow} rows and {nCol} columns')
train_customers.head(3)

In [ ]:
def calc_age(year):
    if len(str(year))==2:
        if str(year).startswith('0'):
            year = '20'+str(year)
        else:
            year = '19'+str(year)
        year = int(year)
    if year is None:
        return np.nan
    return 2020-year

train_customers.loc[:, 'age'] = train_customers['dob'].apply(calc_age)
train_customers[train_customers['age']<16]

In [ ]:
ages = train_customers[train_customers['age']>16]
ages = ages[ages['age']<110]
ages['age'].dropna().hist(bins=20, label='customers by age')